# Кузнецов Михаил Пи19-4


# Биматричные игры



### Импортирование нужных библиотек

In [1]:
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, LpMaximize
import pandas as pd
import nashpy as nh

#### Метод для ввода пользователем матрицы весовых коэффициентов и названия стратегий с клавиатуры

In [2]:
def enterCoef(n, m):
    game_matrix_A = [ [0]*m for i in range(n) ]
    game_matrix_B = [ [0]*m for i in range(n) ]
    names_of_strategies_a = [0] * n
    names_of_strategies_b = [0] * m
    # ввод стратегий игрока А
    for i in range(n):
        name = input(f"Введите название  {i+1} стратегии игрока А: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_a[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
     # ввод стратегий игрока Б
    for i in range(m):
        name = input(f"Введите название  {i+1} стратегии игрока Б: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_b[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
    print("Заполните матрицу для игрока А")        
    for i in range(n):
        for j in range(m):
            num = int(input(f"Введите элемент строки {i+1} столбца {j+1}: "))
            if num > 0:
                game_matrix_A[i][j] = num
            else:
                raise ValueError
    print("Заполните матрицу для игрока B")        
    for i in range(n):
        for j in range(m):
            num = int(input(f"Введите элемент строки {i+1} столбца {j+1}: "))
            if num > 0:
                game_matrix_B[i][j] = num
            else:
                raise ValueError
            
    return (np.array(game_matrix_A),np.array(game_matrix_B), names_of_strategies_a, names_of_strategies_b)

#### Метод для ввода пользователем названия стратегий с клавиатуры и случайная генерация матрицы весовых коэффициентов

In [3]:
def randomCoef(n,m):
    names_of_strategies_a = [0] * n
    names_of_strategies_b = [0] * m
    
    for i in range(n):
        name = input(f"Введите название  {i+1} стратегии игрока А: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_a[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
            
    for i in range(m):
        name = input(f"Введите название  {i+1} стратегии игрока Б: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_b[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError 
            
    game_matrix_A = np.random.randint(1, 500, size = (n,m))
    game_matrix_B = np.random.randint(1, 500, size = (n,m))
    return (game_matrix_A, game_matrix_B, names_of_strategies_a, names_of_strategies_b)

#### Метод для считывания из csv

In [4]:
def readFromScv(path1 = "data/dataA.csv", path2 = "data/dataB.csv"):
    main_matrix = pd.read_csv(path1)
    names_of_strategies_a = list(main_matrix['Names_a'])
    main_matrix.drop('Names_a', inplace=True, axis=1)
    game_matrix = None
    names_of_strategies_b = list(main_matrix.columns)
    if sum(main_matrix.isnull().sum()):
        print("Содержатся пропуски!")
        raise ValueError
    else:
        if sum(main_matrix.dtypes == object) > 0:
            print("Содержится строка в файле!")
            raise ValueError
        elif (main_matrix <= 0).any().any() == True:
            print("Содержится отрицательное или нулевое значение!")
            raise ValueError
        else:
            game_matrix_A = main_matrix.to_numpy()
    
    
    main_matrix = pd.read_csv(path2)
    names_of_strategies_a = list(main_matrix['Names_a'])
    main_matrix.drop('Names_a', inplace=True, axis=1)
    game_matrix = None
    names_of_strategies_b = list(main_matrix.columns)
    if sum(main_matrix.isnull().sum()):
        print("Содержатся пропуски!")
        raise ValueError
    else:
        if sum(main_matrix.dtypes == object) > 0:
            print("Содержится строка в файле!")
            raise ValueError
        elif (main_matrix <= 0).any().any() == True:
            print("Содержится отрицательное или нулевое значение!")
            raise ValueError
        else:
            game_matrix_B = main_matrix.to_numpy()
    
            
    return (game_matrix_A, game_matrix_B, names_of_strategies_a, names_of_strategies_b)

In [15]:
class BimatrixGame:
    
    def __init__(self, game_matrix_A, game_matrix_B, names_strategies_A, names_strategies_B):
        self.game_matrix_A = game_matrix_A
        self.game_matrix_B = game_matrix_B
        self.names_strategies_A = names_strategies_A
        self.names_strategies_B = names_strategies_B
        
        self.pure_A = None
        self.pure_B = None
        self.cost_pure_A = None
        self.cost_pure_B = None
        self.total_pure_cost = None
        
        #для смешанных стратегий
        self.mixed_A = None
        self.mixed_B = None
        self.cost_mixed_A = None
        self.cost_mixed_B = None
        self.total_mixed_cost = None
        self.nash_count = None
        
    
    def simplex_method(self, matrix, F_type, C_type_f):
        model = LpProblem("Game", F_type)
        variables = []

        for i in range(matrix.shape[0]):
            v = LpVariable(f'v{i+1}', lowBound=0)
            variables.append(v)
        
        variables_np = np.array(variables)

        del variables
        
        function = (variables_np).sum()
        model += function
        
        all_constraints = C_type_f((matrix * variables_np).sum(axis=1), 1)
        for c in all_constraints:
            model += c        
        model.solve()
        optimal = np.array(list(map(lambda p: p.value(), model.variables())))
        
        return optimal / optimal.sum()
    
    def solve(self):
        best_A = self.game_matrix_A.max(axis=0)
        worst_B = self.game_matrix_B.min(axis=0)
        
        worst_from_best_A = best_A.argmin()
        best_from_worst_B = worst_B.argmax()

        
        c_p_A = best_A[worst_from_best_A]
        c_p_B = worst_B[best_from_worst_B]
        
        
        mixed_A = self.simplex_method(self.game_matrix_B, LpMinimize, np.vectorize(lambda a, b: a >= b))
        mixed_B = self.simplex_method(self.game_matrix_A, LpMaximize, np.vectorize(lambda x1, x2: x1 <= x2))

        c_m_A = ((mixed_A * self.game_matrix_A.T).T * mixed_B).sum()
        c_m_B = ((mixed_A * self.game_matrix_B.T).T * mixed_B).sum()

        nash = list(nh.Game(self.game_matrix_A, self.game_matrix_B).support_enumeration())
        
        #для чистых стратегий
        self.pure_A = worst_from_best_A + 1
        self.pure_B = best_from_worst_B + 1
        self.cost_pure_A = c_p_A
        self.cost_pure_B = c_p_B
        self.total_pure_cost = c_p_A + c_p_B
        
        #для смешанных стратегий
        self.mixed_A = mixed_A
        self.mixed_B = mixed_B
        self.cost_mixed_A = c_m_A
        self.cost_mixed_B = c_m_B
        self.total_mixed_cost = c_m_A + c_m_B
        self.nash_count = len(nash)
    
    def __str__(self):
        return "Результаты решения\n" + "".join([
        "mixed strat of А: {}\n".format(self.mixed_A),
        "mixed strat of Б: {}\n".format(self.mixed_B),
        "pure strat of А: {}\n".format(self.pure_A),
        "pure strat of Б: {}\n".format(self.pure_B),
        "mixed cost of А: {}\n".format(self.cost_mixed_A),
        "mixed cost of Б: {}\n".format(self.cost_mixed_B),
        "pure cost of А: {}\n".format(self.cost_pure_A),
        "pure cost of Б: {}\n".format(self.cost_pure_B),
        "total cost mixed = {}\n".format(self.total_mixed_cost),
        "total pure cost = {}\n".format(self.total_pure_cost),
        "Nash points count = {}\n".format(self.nash_count)])

In [17]:
games = {1:"Ввод с клавиатуры", 2:"Случайная генерация", 3:"Чтение из csv"}

print(f"Выберите один из 3 вариантов получения данных \n Доступные варианты получения данных: {games}")
print()

game_matrix = None
game_names = None

try: 
    choice = int(input("Введите число для получения данных (от 1 до 3): "))
    
    if (choice > 3) or (choice <= 0):
        print("Введите число от 1 до 3!")
    
    else:
        print(f"Выбранный вариант получения данных: {games[choice]}")
        
        if choice == 1:
            try:
                n = int(input("Введите кол-во стратегий для игрока А: "))
                m = int(input("Введите кол-во стратегий для игрока Б: "))
                
                if (n > 0) and (m > 0):
                    game_matrix_A, game_matrix_B, names_of_strategies_a, names_of_strategies_b  = enterCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                 print("Прервано пользователем")
        elif choice == 2:
            try:
                n = int(input("Введите кол-во стратегий для игрока А: "))
                m = int(input("Введите кол-во стратегий для игрока Б: "))
                
                if (n > 0) and (m > 0):
                    game_matrix_A, game_matrix_B, names_of_strategies_a, names_of_strategies_b  = randomCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
        else:
            try:
                game_matrix_A, game_matrix_B, names_of_strategies_a, names_of_strategies_b = readFromScv()
            except FileNotFoundError:
                print("Файл не найден!")
            except ValueError:
                print("Измените значения ячеек в файле!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
            
except ValueError:
    print("Неправильный ввод!")
except KeyboardInterrupt:
    print("Прервано пользователем")

Выберите один из 3 вариантов получения данных 
 Доступные варианты получения данных: {1: 'Ввод с клавиатуры', 2: 'Случайная генерация', 3: 'Чтение из csv'}

Введите число для получения данных (от 1 до 3): 1
Выбранный вариант получения данных: Ввод с клавиатуры
Введите кол-во стратегий для игрока А: 3
Введите кол-во стратегий для игрока Б: 3
Введите название  1 стратегии игрока А: Телефоны
Введите название  2 стратегии игрока А: Ноутбуки
Введите название  3 стратегии игрока А: ПК
Введите название  1 стратегии игрока Б: Телефоны
Введите название  2 стратегии игрока Б: Наушники
Введите название  3 стратегии игрока Б: Ноутбуки
Заполните матрицу для игрока А
Введите элемент строки 1 столбца 1: 300
Введите элемент строки 1 столбца 2: 280
Введите элемент строки 1 столбца 3: 230
Введите элемент строки 2 столбца 1: 200
Введите элемент строки 2 столбца 2: 180
Введите элемент строки 2 столбца 3: 130
Введите элемент строки 3 столбца 1: 240
Введите элемент строки 3 столбца 2: 380
Введите элемент ст

In [18]:
my_game = BimatrixGame(game_matrix_A, game_matrix_B, names_of_strategies_a, names_of_strategies_b)
my_game.solve()
print(my_game)

Результаты решения
mixed strat of А: [0. 0. 1.]
mixed strat of Б: [0.73913043 0.         0.26086957]
pure strat of А: 1
pure strat of Б: 3
mixed cost of А: 281.73913045438184
mixed cost of Б: 286.9565217330057
pure cost of А: 300
pure cost of Б: 210
total cost mixed = 568.6956521873875
total pure cost = 510
Nash points count = 1

